In [1]:
import textwrap
from datetime import datetime, timedelta
from typing import List, Dict, Any

import dotenv
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate
from langchain.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import MessagesPlaceholder
from langchain_openai import ChatOpenAI

![](images/langchain_agent.png)

In [2]:
# API 키 설정
dotenv.load_dotenv()

True

In [3]:
# 검색 대상 인물 리스트
AI_LEADERS = [
    "Andrew Ng", "Geoffrey Hinton", "Yann LeCun", "Demis Hassabis",
    "Fei-Fei Li", "Ian Goodfellow", "Sam Altman", "Yoshua Bengio",
    "Sebastian Thrun", "Lex Fridman"
]

IT_CEOS = [
    "Elon Musk", "Sundar Pichai", "Satya Nadella", "Tim Cook",
    "Mark Zuckerberg", "Andy Jassy", "Lisa Su", "Marc Benioff",
    "Alexander Caedmon Karp"
]

# 현재 날짜와 일주일 전 날짜 계산
current_date = datetime.now()
one_week_ago = current_date - timedelta(days=7)
date_range = f"{one_week_ago.strftime('%Y-%m-%d')} to {current_date.strftime('%Y-%m-%d')}"

In [4]:
# @tool 데코레이터를 사용한 도구 정의
@tool
# 검색 도구 정의
def search_ai_news(query: str) -> List[Dict[str, Any]]:
    """
    Tavily Search API를 사용하여 뉴스 기사를 검색합니다.
    """
    search_tool = TavilySearchResults(
        max_results=10,
        search_depth="advanced",
        include_answer=False,
        include_raw_content=True,
        # include_domains=[
        #     "techcrunch.com", "wired.com", "theverge.com",
        #     "venturebeat.com", "technologyreview.com",
        #     "zdnet.com", "cnet.com", "engadget.com",
        #     "reuters.com", "bloomberg.com", "cnbc.com"
        # ],
        time_range='week'
    )
    print(f"검색 쿼리: {query}")
    results = search_tool.invoke(query)

    # 최대 5개 기사 반환
    top_results = results[:5]
    print(f"검색된 기사 수: {len(top_results)}")
    return top_results

In [5]:
@tool
def create_blog_post(articles: List[Dict[str, str]]) -> str:
    """
    기사들을 바탕으로 블로그 포스트를 생성합니다.

    Args:
        articles: list of articles
            - url: link to original article
            - title: article title
            - content: article content

    Returns:
        마크다운 형식의 블로그 포스트
    """
    print("블로그 포스트 작성 중...")

    # 기사 요약 목록을 문자열로 변환
    articles_text = ""
    for i, article in enumerate(articles, 1):
        articles_text += f"기사 {i} - 제목: {article['title']}\n"
        articles_text += f"내용: {article['content']}\n"
        articles_text += f"출처: {article['url']}\n\n"

    # 블로그 포스트 생성 프롬프트
    blog_template = """
    You are a friendly and professional AI blog writer. Please write a blog post about AI technology trends based on the following articles.

    Today's date: {current_date}

    Articles:
    {articles_text}

    Blog post writing guidelines:

    1. Write a title that clearly expresses the main content
    2. Include the main content of each article
    3. Write a separate section for each article
    4. Provide a brief conclusion and provide food for thought for the reader at the end
    5. Write in Markdown format (use #, ##, ### in the title, include links, etc.)

    Please write the blog post in Korean.
    """

    # ChatOpenAI를 사용하여 블로그 작성
    blog_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.2)
    response = blog_llm.invoke(blog_template.format(
        current_date=current_date.strftime("%Y년 %m월 %d일"),
        articles_text=articles_text
    ))

    return response.content

In [6]:
def main():
    print("AI 기술 트렌드 관련 인기 기사 검색 및 블로그 포스트 생성 시작...")

    # 1. LLM 초기화
    agent_llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

    # 2. 사용할 도구 목록 정의
    # tools = [search_ai_news, summarize_article, create_blog_post]
    tools = [search_ai_news, create_blog_post]

    # 3. 프롬프트 생성
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
        You are an expert in analyzing the latest news on AI technology trends and writing blog posts based on them.

        Find 3 popular articles on AI technology trends from the past week, analyze them, and write a friendly-toned blog post in Markdown format based on them.

        Follow these steps to complete the task:
        1. Use the search_ai_news tool to search for relevant popular articles.
        2. Select the 3 most relevant articles from the search results.
        3. Use the create_blog_post tool to write a blog post based on the articles.

        The final output should be a blog post in Markdown format written in Korean.
        Please avoid using overly provocative words.
        """),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ])

    # 4. 에이전트 생성
    agent = create_tool_calling_agent(agent_llm, tools, prompt)

    # 5. 에이전트 실행기 생성
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        handle_parsing_errors=True
    )

    # 6. 에이전트 실행
    print("에이전트 실행 중...")

    agent_input = f"Please search for popular articles on AI technology trends over the past week ({date_range}) and write a blog post based on the articles you found."

    result = agent_executor.invoke({"input": agent_input})
    blog_post = result["output"]

    # 7. 블로그 포스트 저장
    output_file = f"ai_trends_blog_{current_date.strftime('%Y%m%d')}.md"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(blog_post)

    print(f"\n블로그 포스트가 성공적으로 생성되었습니다: {output_file}")
    print("\n블로그 포스트 미리보기:")
    print("=" * 80)
    print(textwrap.shorten(blog_post, width=1000, placeholder="..."))
    print("=" * 80)

    return blog_post

In [7]:
# 실행
blog_post = main()
print("프로그램 실행 완료")

AI 기술 트렌드 관련 인기 기사 검색 및 블로그 포스트 생성 시작...
에이전트 실행 중...


> Entering new AgentExecutor chain...

Invoking: `search_ai_news` with `{'query': 'AI technology trends 2025-04-30 to 2025-05-07'}`


검색 쿼리: AI technology trends 2025-04-30 to 2025-05-07
검색된 기사 수: 5
[{'title': 'Latest AI Industry Developments and Trends in 2025', 'url': 'https://news.gab.ai/latest-ai-industry-developments-and-trends-in-2025/', 'content': 'Apple and Google are at the forefront of the latest AI trends in 2025. Apple’s recent AI-driven updates across its ecosystem, released on May 2nd, are revolutionizing user experiences. These enhancements, like advanced AI photo editing, predictive text AI, and smart healthcare features, are designed to maximize productivity and convenience for Apple users. Google continues to push boundaries with AI technology updates. The launch of AI Mode for all Labs users in the United States has made [...] Recent research highlights a rapid rise in AI adoption in manufacturing industry proce